## Movie Reccomendation

#### Reccomending movies based on your favorite movie 

#### Importing neccesary libraries

In [14]:
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#### Loading dataset

In [15]:
data = pd.read_csv("E:/ML projects/Workspace/dataset/Movies Recommendation.csv")

#### Dataset Overview

In [16]:
data.head()

,Movie_ID,Movie_Title,Movie_Genre,Movie_Language,Movie_Budget,Movie_Popularity,Movie_Release_Date,Movie_Revenue,Movie_Runtime,Movie_Vote,...,Movie_Homepage,Movie_Keywords,Movie_Overview,Movie_Production_House,Movie_Production_Country,Movie_Spoken_Language,Movie_Tagline,Movie_Cast,Movie_Crew,Movie_Director
0,1,Four Rooms,Crime Comedy,en,4000000,22.876230,09-12-1995,4300000,98.0,6.5,...,NaN,hotel new year's eve witch bet hotel room,It's Ted the Bellhop's first night on the job....,"[{""name"": ""Miramax Films"", ""id"": 14}, {""name"":...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]",Twelve outrageous guests. Four scandalous requ...,Tim Roth Antonio Banderas Jennifer Beals Madon...,"[{'name': 'Allison Anders', 'gender': 1, 'depa...",Allison Anders
1,2,Star Wars,Adventure Action Science Fiction,en,11000000,126.393695,25-05-1977,775398007,121.0,8.1,...,http://www.starwars.com/films/star-wars-episod...,android galaxy hermit death star lightsaber,Princess Leia is captured and held hostage by ...,"[{""name"": ""Lucasfilm"", ""id"": 1}, {""name"": ""Twe...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]","A long time ago in a galaxy far, far away...",Mark Hamill Harrison Ford Carrie Fisher Peter ...,"[{'name': 'George Lucas', 'gender': 2, 'depart...",George Lucas
2,3,Finding Nemo,Animation Family,en,94000000,85.688789,30-05-2003,940335536,100.0,7.6,...,http://movies.disney.com/finding-nemo,father son relationship harbor underwater fish...,"Nemo, an adventurous young clownfish, is unexp...","[{""name"": ""Pixar Animation Studios"", ""id"": 3}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]","There are 3.7 trillion fish in the ocean, they...",Albert Brooks Ellen DeGeneres Alexander Gould ...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
3,4,Forrest Gump,Comedy Drama Romance,en,55000000,138.133331,06-07-1994,677945399,142.0,8.2,...,NaN,vietnam veteran hippie mentally disabled runni...,A man with a low IQ has accomplished great thi...,"[{""name"": ""Paramount Pictures"", ""id"": 4}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]","The world will never be the same, once you've ...",Tom Hanks Robin Wright Gary Sinise Mykelti Wil...,"[{'name': 'Alan Silvestri', 'gender': 2, 'depa...",Robert Zemeckis
4,5,American Beauty,Drama,en,15000000,80.878605,15-09-1999,356296601,122.0,7.9,...,http://www.dreamworks.com/ab/,male nudity female nudity adultery midlife cri...,"Lester Burnham, a depressed suburban father in...","[{""name"": ""DreamWorks SKG"", ""id"": 27}, {""name""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]",Look closer.,Kevin Spacey Annette Bening Thora Birch Wes Be...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4760 entries, 0 to 4759
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Movie_ID                  4760 non-null   int64  
 1   Movie_Title               4760 non-null   object 
 2   Movie_Genre               4760 non-null   object 
 3   Movie_Language            4760 non-null   object 
 4   Movie_Budget              4760 non-null   int64  
 5   Movie_Popularity          4760 non-null   float64
 6   Movie_Release_Date        4760 non-null   object 
 7   Movie_Revenue             4760 non-null   int64  
 8   Movie_Runtime             4758 non-null   float64
 9   Movie_Vote                4760 non-null   float64
 10  Movie_Vote_Count          4760 non-null   int64  
 11  Movie_Homepage            1699 non-null   object 
 12  Movie_Keywords            4373 non-null   object 
 13  Movie_Overview            4757 non-null   object 
 14  Movie_Pr

In [18]:
data.describe()

,Movie_ID,Movie_Budget,Movie_Popularity,Movie_Revenue,Movie_Runtime,Movie_Vote,Movie_Vote_Count
count,4760.000000,4.760000e+03,4760.000000,4.760000e+03,4758.000000,4760.000000,4760.000000
mean,2382.566387,2.920129e+07,21.599510,8.263743e+07,107.184111,6.113866,692.508403
std,1377.270159,4.075620e+07,31.887919,1.630554e+08,21.960332,1.141294,1235.007337
min,1.000000,0.000000e+00,0.000372,0.000000e+00,0.000000,0.000000,0.000000
25%,1190.750000,9.257500e+05,4.807074,0.000000e+00,94.000000,5.600000,55.000000
50%,2380.500000,1.500000e+07,13.119058,1.944716e+07,104.000000,6.200000,238.000000
75%,3572.250000,4.000000e+07,28.411929,9.341276e+07,118.000000,6.800000,740.250000
max,4788.000000,3.800000e+08,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


In [19]:
data.shape

(4760, 21)

In [20]:
data.columns

Index(['Movie_ID', 'Movie_Title', 'Movie_Genre', 'Movie_Language',
       'Movie_Budget', 'Movie_Popularity', 'Movie_Release_Date',
       'Movie_Revenue', 'Movie_Runtime', 'Movie_Vote', 'Movie_Vote_Count',
       'Movie_Homepage', 'Movie_Keywords', 'Movie_Overview',
       'Movie_Production_House', 'Movie_Production_Country',
       'Movie_Spoken_Language', 'Movie_Tagline', 'Movie_Cast', 'Movie_Crew',
       'Movie_Director'],
      dtype='object')

#### Feature Defining

In [21]:
data_features=data[['Movie_Genre','Movie_Language' , 'Movie_Tagline', 'Movie_Cast',
       'Movie_Director']].fillna('')

In [22]:
data_features.isnull().sum()

Movie_Genre       0
Movie_Language    0
Movie_Tagline     0
Movie_Cast        0
Movie_Director    0
dtype: int64

In [23]:
data_features.shape

(4760, 5)

In [24]:
data_features

,Movie_Genre,Movie_Language,Movie_Tagline,Movie_Cast,Movie_Director
0,Crime Comedy,en,Twelve outrageous guests. Four scandalous requ...,Tim Roth Antonio Banderas Jennifer Beals Madon...,Allison Anders
1,Adventure Action Science Fiction,en,"A long time ago in a galaxy far, far away...",Mark Hamill Harrison Ford Carrie Fisher Peter ...,George Lucas
2,Animation Family,en,"There are 3.7 trillion fish in the ocean, they...",Albert Brooks Ellen DeGeneres Alexander Gould ...,Andrew Stanton
3,Comedy Drama Romance,en,"The world will never be the same, once you've ...",Tom Hanks Robin Wright Gary Sinise Mykelti Wil...,Robert Zemeckis
4,Drama,en,Look closer.,Kevin Spacey Annette Bening Thora Birch Wes Be...,Sam Mendes
...,...,...,...,...,...
4755,Horror,en,The hot spot where Satan's waitin'.,Lisa Hart Carroll Michael Des Barres Paul Drak...,Pece Dingo
4756,Comedy Family Drama,en,It’s better to stand out than to fit in.,Roni Akurati Brighton Sharbino Jason Lee Anjul...,Frank Lotito
4757,Thriller Drama,en,She never knew it could happen to her...,Nicole Smolen Kim Baldwin Ariana Stephens Brys...,Jaco Booyens
4758,Family,en,,,


In [25]:
x = data_features['Movie_Genre'] + ' ' + data_features['Movie_Language'] + ' ' + data_features['Movie_Tagline'] + ' ' + data_features['Movie_Cast'] + ' ' + data_features['Movie_Director']

In [26]:
x

0       Crime Comedy en Twelve outrageous guests. Four...
1       Adventure Action Science Fiction en A long tim...
2       Animation Family en There are 3.7 trillion fis...
3       Comedy Drama Romance en The world will never b...
4       Drama en Look closer. Kevin Spacey Annette Ben...
                              ...                        
4755    Horror en The hot spot where Satan's waitin'. ...
4756    Comedy Family Drama en It’s better to stand ou...
4757    Thriller Drama en She never knew it could happ...
4758                                         Family en   
4759    Documentary en  Tony Oppedisano Simon Napier-Bell
Length: 4760, dtype: object

In [27]:
x.shape

(4760,)

#### Features text conversion to Vectors

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [29]:
tfidf = TfidfVectorizer()

In [30]:
x = tfidf.fit_transform(x)

In [31]:
x.shape

(4760, 14544)

In [32]:
print(x)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 102396 stored elements and shape (4760, 14544)>
  Coords	Values
  (0, 2988)	0.07446884432177164
  (0, 2732)	0.051730815699019154
  (0, 4107)	0.027280461481123095
  (0, 13185)	0.2015020198904683
  (0, 9701)	0.2145997856024667
  (0, 5451)	0.2145997856024667
  (0, 4733)	0.16865838280597267
  (0, 11390)	0.22499606711872763
  (0, 10756)	0.22499606711872763
  (0, 534)	0.09527044998711498
  (0, 9608)	0.09875245379952459
  (0, 7846)	0.1864309434460892
  (0, 1254)	0.22499606711872763
  (0, 6304)	0.17714931055968303
  (0, 5996)	0.22624907630503105
  (0, 4601)	0.13821899743990088
  (0, 3256)	0.13865730529428927
  (0, 9606)	0.10836821569911338
  (0, 12786)	0.1142679360474606
  (0, 6660)	0.17167838519837808
  (0, 14067)	0.12201445975996707
  (0, 4690)	0.09875245379952459
  (0, 14100)	0.2015020198904683
  (0, 9308)	0.1317269571511156
  (0, 14346)	0.15471987340945187
  :	:
  (4757, 12863)	0.08168494065714148
  (4757, 7097)	0.18654764519142

#### Similarity score using cosine similarity

In [33]:
similarity_score = cosine_similarity(x)

In [34]:
similarity_score

array([[1.        , 0.02174536, 0.05541309, ..., 0.00101682, 0.00879419,
        0.00183888],
       [0.02174536, 1.        , 0.01482936, ..., 0.00134535, 0.01163559,
        0.00243302],
       [0.05541309, 0.01482936, 1.        , ..., 0.00132677, 0.11042311,
        0.00239941],
       ...,
       [0.00101682, 0.00134535, 0.00132677, ..., 1.        , 0.01201531,
        0.00251242],
       [0.00879419, 0.01163559, 0.11042311, ..., 0.01201531, 1.        ,
        0.02172927],
       [0.00183888, 0.00243302, 0.00239941, ..., 0.00251242, 0.02172927,
        1.        ]])

In [35]:
similarity_score.shape

(4760, 4760)

#### Taking movie name as Input from user

In [36]:
Fav_movie_name = input('Enter your favourite movie name : ')

In [37]:
all_movies_title_list = data['Movie_Title'].tolist()

In [38]:
Movie_Recommendtion = difflib.get_close_matches(Fav_movie_name, all_movies_title_list)
print(Movie_Recommendtion)

['Gattaca']


In [40]:
Close_Match= Movie_Recommendtion[0]
print(Close_Match)

Gattaca


In [41]:
Index_of_Close_Match_movie = data[data.Movie_Title == Close_Match]['Movie_ID'].values[0]
print(Index_of_Close_Match_movie)

348


In [42]:
Recommendation_score = list(enumerate(similarity_score[Index_of_Close_Match_movie]))
print(Recommendation_score)

[(0, np.float64(0.04888071448144665)), (1, np.float64(0.0014644407592683457)), (2, np.float64(0.0077787786053831924)), (3, np.float64(0.04766148844398239)), (4, np.float64(0.006370195077390811)), (5, np.float64(0.005586954993153518)), (6, np.float64(0.0017155188270223311)), (7, np.float64(0.01672416920843879)), (8, np.float64(0.006659349729808142)), (9, np.float64(0.008376727220253255)), (10, np.float64(0.056758762481430754)), (11, np.float64(0.01521413424643702)), (12, np.float64(0.014004303675914164)), (13, np.float64(0.018994274672643252)), (14, np.float64(0.001461989520587099)), (15, np.float64(0.005113442344290502)), (16, np.float64(0.003262627728994768)), (17, np.float64(0.0017374875956276356)), (18, np.float64(0.012129841457919885)), (19, np.float64(0.005503734600412388)), (20, np.float64(0.006276866476577799)), (21, np.float64(0.005883028549161737)), (22, np.float64(0.001667324431527912)), (23, np.float64(0.0062478542351292864)), (24, np.float64(0.019405470171241154)), (25, np.

In [43]:
len(Recommendation_score)

4760

#### Sorting all movies based on reccomendation score with respect to Fav movie

In [44]:
Sorted_Similar_Movies = sorted(Recommendation_score, key = lambda x:x[1], reverse = True)
print(Sorted_Similar_Movies)

[(348, np.float64(1.0)), (4479, np.float64(0.19767734045126845)), (934, np.float64(0.1974078888766923)), (494, np.float64(0.18027157748863992)), (657, np.float64(0.17398638958207738)), (1974, np.float64(0.1710589317069595)), (2725, np.float64(0.16956285229272455)), (2920, np.float64(0.15211975735443756)), (1241, np.float64(0.14983282367077447)), (120, np.float64(0.14455629836198905)), (2037, np.float64(0.14207344740290265)), (2142, np.float64(0.1418631961200385)), (1500, np.float64(0.13769262046047145)), (1946, np.float64(0.13379631727109836)), (3731, np.float64(0.13331735829451746)), (184, np.float64(0.1303495406740318)), (1685, np.float64(0.12872836160217863)), (2493, np.float64(0.12686667467056467)), (749, np.float64(0.12504390886403763)), (936, np.float64(0.12383447277023563)), (4533, np.float64(0.12223079666754812)), (911, np.float64(0.11785147805530433)), (626, np.float64(0.11309906775664164)), (2977, np.float64(0.11300393826260915)), (2306, np.float64(0.11211792472957334)), (209

#### Suggesting top 30 movies

In [45]:
print('Top 30 Movies suggested for you : \n')

i = 1

for movie in Sorted_Similar_Movies:
    index = movie[0]
    title_from_index = data[data.index==index]['Movie_Title'].values[0]
    if (i<31):
        print(i, '.',title_from_index)
        i+=1

Top 30 Movies suggested for you : 

1 . Gandhi
2 . The Theory of Everything
3 . A Bridge Too Far
4 . Miss Congeniality
5 . Mr. Holland's Opus
6 . Sweet Home Alabama
7 . The Road
8 . Agora
9 . The Longest Day
10 . Oliver Twist
11 . Sexy Beast
12 . Cry Freedom
13 . Glory Road
14 . The New World
15 . The Night Visitor
16 . Schindler's List
17 . Shadowlands
18 . Jack Brooks: Monster Slayer
19 . The Bounty
20 . The Great Escape
21 . The Walk
22 . The Hunting Party
23 . The Elephant Man
24 . The Messenger
25 . The Wackness
26 . Three Men and a Baby
27 . Nixon
28 . The Lost World: Jurassic Park
29 . Shutter Island
30 . Jurassic Park


#### Top 10 movies Reccomending system

In [47]:
Movie_Name = input(' Enter your favourite movie name : ')
print("your favorite movie entered : ",Movie_Name)
list_of_all_titles = data['Movie_Title'].tolist()
Find_Close_Match = difflib.get_close_matches(Movie_Name, list_of_all_titles)
Close_Match = Find_Close_Match[0]
Index_of_Movie = data[data.Movie_Title == Close_Match]['Movie_ID'].values[0]
Recommendation_score = list(enumerate(similarity_score[Index_of_Movie]))
Sorted_Similar_Movies = sorted(Recommendation_score, key = lambda x:x[1], reverse = True)
print('Top 10 Movies suggested for you : \n')
i = 1
for movie in Sorted_Similar_Movies:
  index= movie[0]
  title_from_index= data[data.Movie_ID==index]['Movie_Title'].values
  if (i<11):
    print(i, '.',title_from_index)
    i+=1

your favorite movie entered :  avatar
Top 10 Movies suggested for you : 

1 . ['Avatar']
2 . ['Donnie Darko']
3 . ['The Girl on the Train']
4 . ['Freaky Friday']
5 . ['Repo! The Genetic Opera']
6 . ['The Godfather']
7 . ['New Nightmare']
8 . ['Rollerball']
9 . ['A Prairie Home Companion']
10 . ['The Merchant of Venice']
